In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

texts_file = '/content/drive/My Drive/Магистерская диссертация/4 семестр/arctic_material_result.xlsx'

# Часть 1: Подготовка данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Загрузка данных
texts_file = '/content/drive/My Drive/Магистерская диссертация/4 семестр/arctic_material_result.xlsx'
df = pd.read_excel(texts_file)

# Формирование полного текста
df['full_text'] = df['title'].fillna('') + ".\n" + df['content'].fillna('')

# Проверка количества уникальных текстов до группировки
print("Уникальных текстов в исходных данных:", df['full_text'].nunique())

# Группировка по уникальным текстам и объединение тематик
aggregated = df.groupby('full_text')['theme_title'].apply(lambda x: list(set(x))).reset_index()

# Определяем уникальные темы
theme_list = sorted(df['theme_title'].unique())
if 'Not Classified' in theme_list:
    theme_list.remove('Not Classified')  # Исключаем 'Not Classified' из списка тематик

# Создаем DataFrame с бинарными метками
for theme in theme_list:
    aggregated[theme] = aggregated['theme_title'].apply(lambda x: 1 if theme in x else 0)

# Добавляем столбцы с 0 для 'Not Classified'
aggregated.loc[aggregated['theme_title'].apply(lambda x: 'Not Classified' in x), theme_list] = 0

# Удаляем ненужный столбец
aggregated.drop(columns=['theme_title'], inplace=True)

# Выгрузка результата в Excel
output_file = '/content/drive/My Drive/Магистерская диссертация/4 семестр/prepared_data.xlsx'
aggregated.to_excel(output_file, index=False)

X = aggregated['full_text']
Y = aggregated[theme_list]

# Разделение на обучающую и тестовую выборки (80/20)
random_state = 42  # фиксированный random_state для всех методов
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=random_state)
print("Train: " + str(len(X_train)))
print("Test: " + str(len(X_test)))


## Инициализация таблиц для метрик

In [ ]:
import pandas as pd
import os

# Инициализация списка методов классификации и тематик
methods = ['Decision Tree', 'Zero-Shot Binary', 'Zero-Shot Binary Optimal', 'Zero-Shot Multi', 'SVM', 'Naive Bayes', 'Logistic Regression']

# Папка для сохранения результатов
output_dir = '/content/drive/My Drive/Магистерская диссертация/4 семестр/Metrics results'
os.makedirs(output_dir, exist_ok=True)

# Создание пустых DataFrame с фиксированными методами и тематиками
accuracy_results = pd.DataFrame({'method': methods})
precision_results = pd.DataFrame({'method': methods})
recall_results = pd.DataFrame({'method': methods})
f1_results = pd.DataFrame({'method': methods})

for theme in theme_list:
    accuracy_results[theme] = 0.0
    precision_results[theme] = 0.0
    recall_results[theme] = 0.0
    f1_results[theme] = 0.0

print("Инициализированы таблицы для метрик.")
# print(accuracy_results)
# print(precision_results)
# print(recall_results)
# print(f1_results)

# Функция сохранения всех таблиц в Excel
def save_metrics():
    accuracy_results.to_excel(os.path.join(output_dir, 'accuracy_results.xlsx'), index=False)
    precision_results.to_excel(os.path.join(output_dir, 'precision_results.xlsx'), index=False)
    recall_results.to_excel(os.path.join(output_dir, 'recall_results.xlsx'), index=False)
    f1_results.to_excel(os.path.join(output_dir, 'f1_results.xlsx'), index=False)
    print("Таблицы метрик сохранены в Excel.")


## Создание таблицы с метриками (TP, TN, FP, FN)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix

def print_confusion_table(y_true, y_pred, method_name, theme):
    """
    Функция для создания и вывода таблицы с метриками TP, TN, FP, FN.
    Аргументы:
        y_true: истинные значения меток (массив или список)
        y_pred: предсказанные значения меток (массив или список)
        method_name: название метода классификации (строка)
        theme: название тематики (строка)
    """
    # Вычисляем матрицу ошибок
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Создаем таблицу с метриками
    confusion_df = pd.DataFrame(
        [[tp, fp], [fn, tn]],
        columns=['Релевантные (1)', 'Нерелевантные (0)'],
        index=['Найденные (1)', 'Ненайденные (0)']
    )

    print(f"\nМетод: {method_name}, Тематика: {theme}")
    print("Таблица с метриками (TP, TN, FP, FN):")
    print(confusion_df)
    print("\n")


# Часть 2: Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def decision_tree_classifier(X_train, X_test, Y_train, Y_test, theme_list,
                             accuracy_results, precision_results, recall_results, f1_results):
    current_method = 'Decision Tree'
    print("\n" + current_method)

    # Преобразование текстов в признаки
    vectorizer = CountVectorizer()
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    # Обучение и тестирование 9 бинарных классификаторов
    for theme in theme_list:
        # Создание и обучение классификатора
        clf = DecisionTreeClassifier(random_state=42)
        clf.fit(X_train_vec, Y_train[theme])

        # Предсказание и расчет метрик
        Y_pred = clf.predict(X_test_vec)
        accuracy = accuracy_score(Y_test[theme], Y_pred)
        precision = precision_score(Y_test[theme], Y_pred, zero_division=0)
        recall = recall_score(Y_test[theme], Y_pred, zero_division=0)
        f1 = f1_score(Y_test[theme], Y_pred, zero_division=0)

        # Запись метрик в таблицы (по методу и теме)
        accuracy_results.loc[accuracy_results['method'] == current_method, theme] = round(accuracy, 2)
        precision_results.loc[precision_results['method'] == current_method, theme] = round(precision, 2)
        recall_results.loc[recall_results['method'] == current_method, theme] = round(recall, 2)
        f1_results.loc[f1_results['method'] == current_method, theme] = round(f1, 2)

        print(f'Theme: {theme}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

        # Вывод таблицы с метриками
        print_confusion_table(Y_test[theme], Y_pred, current_method, theme)


In [ ]:
# Запуск классификатора
decision_tree_classifier(X_train, X_test, Y_train, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)

# Сохранение метрик в Excel
save_metrics()

### Сохранение модели для библиотеки:

In [ ]:
import os
import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Папка для сохранения моделей
save_dir = '/content/drive/My Drive/Магистерская диссертация/4 семестр/Библиотека/Модели для библиотеки/Decision Tree'
os.makedirs(save_dir, exist_ok=True)

def decision_tree_classifier_and_save(X_train, X_test, Y_train, Y_test, theme_list,
                             accuracy_results, precision_results, recall_results, f1_results):
    current_method = 'Decision Tree'
    print("\n" + current_method)

    # Векторизация текста
    vectorizer = CountVectorizer()
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    # Сохраняем векторизатор
    vectorizer_path = os.path.join(save_dir, f'{current_method}_vectorizer.pkl')
    joblib.dump(vectorizer, vectorizer_path)

    # Обучение и сохранение моделей для каждой темы
    for theme in theme_list:
        print(f'\n--- Тема: {theme} ---')

        # Обучение модели
        clf = DecisionTreeClassifier(random_state=42)
        clf.fit(X_train_vec, Y_train[theme])

        # Сохранение модели
        model_path = os.path.join(save_dir, f'{current_method}_{theme}.pkl')
        joblib.dump(clf, model_path)

        # Предсказание и метрики
        Y_pred = clf.predict(X_test_vec)
        accuracy = accuracy_score(Y_test[theme], Y_pred)
        precision = precision_score(Y_test[theme], Y_pred, zero_division=0)
        recall = recall_score(Y_test[theme], Y_pred, zero_division=0)
        f1 = f1_score(Y_test[theme], Y_pred, zero_division=0)

        # Сохраняем метрики
        accuracy_results.loc[accuracy_results['method'] == current_method, theme] = round(accuracy, 2)
        precision_results.loc[precision_results['method'] == current_method, theme] = round(precision, 2)
        recall_results.loc[recall_results['method'] == current_method, theme] = round(recall, 2)
        f1_results.loc[f1_results['method'] == current_method, theme] = round(f1, 2)

        print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')
        # print_confusion_table(Y_test[theme], Y_pred, current_method, theme)

# Вызов функции
decision_tree_classifier_and_save(X_train, X_test, Y_train, Y_test, theme_list,
                                  accuracy_results, precision_results, recall_results, f1_results)

# Сохранение таблиц с метриками
# save_metrics()


# Часть 3: Zero-Shot

### 3.1.1: Одноклассовая

In [ ]:
from transformers import pipeline
import numpy as np

def zero_shot_binary_classifier(X_test, Y_test, theme_list,
                                accuracy_results, precision_results, recall_results, f1_results):
    current_method = 'Zero-Shot Binary'
    print("\n" + current_method)

    # Загрузка модели zero-shot
    zero_shot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")

    # Цикл по всем тематикам
    for theme in theme_list:
        y_true = Y_test[theme].values  # Истинные метки класса
        y_pred = []

        # Применение модели ко всем текстам тестовой выборки
        for text in X_test:
            result = zero_shot_classifier(text, candidate_labels=[theme])
            # Если модель предсказывает целевой класс с высокой вероятностью (>0.5), считаем, что текст относится к этому классу.
            prediction = int(result["labels"][0] == theme and result["scores"][0] > 0.5)
            y_pred.append(prediction)

        # Расчет метрик
        accuracy = np.mean(np.array(y_pred) == np.array(y_true))
        precision = np.sum((np.array(y_pred) == 1) & (np.array(y_true) == 1)) / np.sum(np.array(y_pred) == 1) if np.sum(np.array(y_pred) == 1) > 0 else 0
        recall = np.sum((np.array(y_pred) == 1) & (np.array(y_true) == 1)) / np.sum(np.array(y_true) == 1) if np.sum(np.array(y_true) == 1) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Запись метрик в таблицы (по методу и теме)
        accuracy_results.loc[accuracy_results['method'] == current_method, theme] = round(accuracy, 2)
        precision_results.loc[precision_results['method'] == current_method, theme] = round(precision, 2)
        recall_results.loc[recall_results['method'] == current_method, theme] = round(recall, 2)
        f1_results.loc[f1_results['method'] == current_method, theme] = round(f1, 2)

        print(f'Theme: {theme}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

        # Вывод таблицы с метриками
        print_confusion_table(y_true, y_pred, current_method, theme)


In [ ]:
# Запуск классификатора
zero_shot_binary_classifier(X_test, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)

# Сохранение метрик в Excel
save_metrics()

### 3.1.2: Одноклассовая оптимальная

#### Код для тестирования на одной тематике с использованием ROC-кривой и AUC для подбора оптимального порога:

In [ ]:
from transformers import pipeline
import numpy as np
from sklearn.metrics import roc_curve, auc, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt

def zero_shot_binary_classifier_auc(X_test, Y_test, theme,
                                    accuracy_results, precision_results, recall_results, f1_results):
    current_method = 'Zero-Shot Binary AUC'
    print("\n" + current_method)

    # Загрузка модели zero-shot
    zero_shot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")

    y_true = Y_test[theme].values  # Истинные метки класса
    y_scores = []

    # Применение модели ко всем текстам тестовой выборки
    for text in X_test:
        result = zero_shot_classifier(text, candidate_labels=[theme])
        # Сохраняем вероятность для целевого класса
        score = result["scores"][0]
        y_scores.append(score)

    # Построение ROC-кривой
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    # Оптимальный порог по максимальному F1-скор
    f1_scores = [f1_score(y_true, [1 if s >= thr else 0 for s in y_scores]) for thr in thresholds]
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx]

    # Прогнозы с оптимальным порогом
    y_pred = [1 if score >= optimal_threshold else 0 for score in y_scores]

    # Метрики
    accuracy = np.mean(np.array(y_pred) == np.array(y_true))
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f'Theme: {theme}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')
    print(f'Optimal threshold: {optimal_threshold}')

    # Построение ROC-кривой
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {theme}')
    plt.legend(loc="lower right")
    plt.show()

    # Вывод таблицы с метриками
    print_confusion_table(y_true, y_pred, current_method, theme)

# Запуск классификатора на одной тематике
zero_shot_binary_classifier_auc(X_test, Y_test, 'Arctic Indigenous Peoples',
                                accuracy_results, precision_results, recall_results, f1_results)


#### Код для подбора порогов

Код, который автоматически вычисляет оптимальные пороги для всех тематик и затем использует их при классификации:

In [ ]:
from transformers import pipeline
import numpy as np
from sklearn.metrics import roc_curve, auc, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt

def find_optimal_thresholds(X_test, Y_test, theme_list):
    optimal_thresholds = {}
    zero_shot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")

    for theme in theme_list:
        y_true = Y_test[theme].values
        y_scores = []

        # Получаем вероятности для всех текстов
        for text in X_test:
            result = zero_shot_classifier(text, candidate_labels=[theme])
            score = result["scores"][0]
            y_scores.append(score)

        # ROC-кривая и AUC
        fpr, tpr, thresholds = roc_curve(y_true, y_scores)
        roc_auc = auc(fpr, tpr)

        # Поиск оптимального порога (максимизация F1-score)
        f1_scores = [f1_score(y_true, [1 if s >= thr else 0 for s in y_scores]) for thr in thresholds]
        optimal_idx = np.argmax(f1_scores)
        optimal_threshold = thresholds[optimal_idx]
        optimal_thresholds[theme] = optimal_threshold

        # Вывод результатов для темы
        print(f'Theme: {theme}, AUC: {roc_auc:.2f}, Optimal threshold: {optimal_threshold:.2f}')

        # # Построение ROC-кривой
        # plt.figure(figsize=(8, 6))
        # plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        # plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        # plt.xlabel('False Positive Rate')
        # plt.ylabel('True Positive Rate')
        # plt.title(f'ROC Curve for {theme}')
        # plt.legend(loc="lower right")
        # plt.show()

    return optimal_thresholds


#### ! Код для классификации с учетом оптимальных порогов:

In [ ]:
from transformers import pipeline
import numpy as np

def zero_shot_binary_classifier_with_thresholds(X_test, Y_test, theme_list, optimal_thresholds,
                                                accuracy_results, precision_results, recall_results, f1_results):
    current_method = 'Zero-Shot Binary Optimal'
    print("\n" + current_method)

    zero_shot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")

    for theme in theme_list:
        y_true = Y_test[theme].values
        y_pred = []

        # Получаем вероятности и применяем оптимальный порог
        for text in X_test:
            result = zero_shot_classifier(text, candidate_labels=[theme])
            score = result["scores"][0]

            # Если оптимальный порог больше среднего оптимального, он используется. Если меньше — берется средний.
            # prediction = int(score >= max(optimal_thresholds[theme], np.mean(list(optimal_thresholds.values()))))

            # Если оптимальный порог больше 0.8, он используется. Если меньше — берется 0.8.
            prediction = int(score >= max(optimal_thresholds[theme], 0.8))

            y_pred.append(prediction)

        # Метрики
        accuracy = np.mean(np.array(y_pred) == np.array(y_true))
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        # Запись метрик в таблицы
        accuracy_results.loc[accuracy_results['method'] == current_method, theme] = round(accuracy, 2)
        precision_results.loc[precision_results['method'] == current_method, theme] = round(precision, 2)
        recall_results.loc[recall_results['method'] == current_method, theme] = round(recall, 2)
        f1_results.loc[f1_results['method'] == current_method, theme] = round(f1, 2)

        print(f'Theme: {theme}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

        # Вывод таблицы с метриками
        print_confusion_table(y_true, y_pred, current_method, theme)



In [ ]:
# Вычисляем оптимальные пороги
optimal_thresholds = find_optimal_thresholds(X_test, Y_test, theme_list)

# Запускаем классификатор с оптимальными порогами
zero_shot_binary_classifier_with_thresholds(X_test, Y_test, theme_list, optimal_thresholds,
                                            accuracy_results, precision_results, recall_results, f1_results)

# Сохранение метрик в Excel
save_metrics()

In [ ]:
print(optimal_thresholds)

In [ ]:
# Вычисляем среднее значение оптимальных порогов
average_threshold = np.mean(list(optimal_thresholds.values()))
print(f'Средний оптимальный порог: {average_threshold:.2f}')


### 3.2: Многоклассовая

In [ ]:
from transformers import pipeline
import numpy as np

def zero_shot_multi_classifier(X_test, Y_test, theme_list,
                               accuracy_results, precision_results, recall_results, f1_results):
    current_method = 'Zero-Shot Multi'
    print("\n" + current_method)

    # Загрузка модели zero-shot
    zero_shot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")

    # Преобразуем список тем в строковый формат для модели
    candidate_labels = list(theme_list)

    # Цикл по всем тематикам
    for theme in theme_list:
        y_true = Y_test[theme].values  # Истинные метки класса
        y_pred = []

        # Применение модели ко всем текстам тестовой выборки
        for text in X_test:
            result = zero_shot_classifier(text, candidate_labels=candidate_labels)

            # Лучше использовать zero-shot как многоклассовую классификацию, где модель сразу оценивает вероятность принадлежности текста ко всем темам одновременно.
            # Если целевая тема оказывается на первом месте по вероятности, считаем текст принадлежащим ей.

            # Проверяем, оказалась ли нужная тема на первом месте среди предсказанных
            predicted_label = result["labels"][0]
            prediction = int(predicted_label == theme)
            y_pred.append(prediction)

        # Расчет метрик
        accuracy = np.mean(np.array(y_pred) == np.array(y_true))
        precision = np.sum((np.array(y_pred) == 1) & (np.array(y_true) == 1)) / np.sum(np.array(y_pred) == 1) if np.sum(np.array(y_pred) == 1) > 0 else 0
        recall = np.sum((np.array(y_pred) == 1) & (np.array(y_true) == 1)) / np.sum(np.array(y_true) == 1) if np.sum(np.array(y_true) == 1) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Запись метрик в таблицы (по методу и теме)
        accuracy_results.loc[accuracy_results['method'] == current_method, theme] = round(accuracy, 2)
        precision_results.loc[precision_results['method'] == current_method, theme] = round(precision, 2)
        recall_results.loc[recall_results['method'] == current_method, theme] = round(recall, 2)
        f1_results.loc[f1_results['method'] == current_method, theme] = round(f1, 2)

        print(f'Theme: {theme}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

        # Вывод таблицы с метриками
        print_confusion_table(y_true, y_pred, current_method, theme)


In [ ]:
# Запуск классификатора
zero_shot_multi_classifier(X_test, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)

# Сохранение метрик в Excel
save_metrics()

# Часть 4: Метод опорных векторов (SVM)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Модель SVM
def svm_classifier(X_train, X_test, Y_train, Y_test, theme_list,
                   accuracy_results, precision_results, recall_results, f1_results):
    current_method = 'SVM'
    print("\n" + current_method)

    # Векторизация текста с помощью TfidfVectorizer
    vectorizer = TfidfVectorizer()
    X_train_vect = vectorizer.fit_transform(X_train)
    X_test_vect = vectorizer.transform(X_test)

    for theme in theme_list:
        # Тренируем и тестируем SVM для каждой тематики
        # Бинарная метка для каждой темы
        y_train_theme = Y_train[theme]
        y_test_theme = Y_test[theme]

        # Создание классификатора SVM
        model = SVC(kernel='linear', random_state=42)

        # Обучение модели
        model.fit(X_train_vect, y_train_theme)

        # Прогнозирование
        y_pred = model.predict(X_test_vect)

        # Метрики
        accuracy = accuracy_score(y_test_theme, y_pred)
        precision = precision_score(y_test_theme, y_pred)
        recall = recall_score(y_test_theme, y_pred)
        f1 = f1_score(y_test_theme, y_pred)

        # Запись метрик в таблицы (по методу и теме)
        accuracy_results.loc[accuracy_results['method'] == current_method, theme] = round(accuracy, 2)
        precision_results.loc[precision_results['method'] == current_method, theme] = round(precision, 2)
        recall_results.loc[recall_results['method'] == current_method, theme] = round(recall, 2)
        f1_results.loc[f1_results['method'] == current_method, theme] = round(f1, 2)

        print(f'Theme: {theme}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

        # Вывод таблицы с метриками
        print_confusion_table(y_test_theme, y_pred, current_method, theme)


In [ ]:
# Запуск классификатора SVM
svm_classifier(X_train, X_test, Y_train, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)

# Сохранение метрик в Excel
save_metrics()

### Сохранение модели для библиотеки:

In [ ]:
import os
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Папка для сохранения моделей
save_dir = '/content/drive/My Drive/Магистерская диссертация/4 семестр/Библиотека/Модели для библиотеки/SVM'
os.makedirs(save_dir, exist_ok=True)

def svm_classifier_and_save(X_train, X_test, Y_train, Y_test, theme_list,
                            accuracy_results, precision_results, recall_results, f1_results):
    current_method = 'SVM'
    print("\n" + current_method)

    # Векторизация текста
    vectorizer = TfidfVectorizer()
    X_train_vect = vectorizer.fit_transform(X_train)
    X_test_vect = vectorizer.transform(X_test)

    # Сохраняем векторизатор
    vectorizer_path = os.path.join(save_dir, f'{current_method}_vectorizer.pkl')
    joblib.dump(vectorizer, vectorizer_path)

    # Обучение и сохранение моделей
    for theme in theme_list:
        print(f'\n--- Тема: {theme} ---')

        y_train_theme = Y_train[theme]
        y_test_theme = Y_test[theme]

        model = SVC(kernel='linear', random_state=42)
        model.fit(X_train_vect, y_train_theme)

        # Сохраняем модель
        model_path = os.path.join(save_dir, f'{current_method}_{theme}.pkl')
        joblib.dump(model, model_path)

        # Предсказания
        y_pred = model.predict(X_test_vect)

        # Метрики
        accuracy = accuracy_score(y_test_theme, y_pred)
        precision = precision_score(y_test_theme, y_pred, zero_division=0)
        recall = recall_score(y_test_theme, y_pred, zero_division=0)
        f1 = f1_score(y_test_theme, y_pred, zero_division=0)

        # Запись в таблицы
        accuracy_results.loc[accuracy_results['method'] == current_method, theme] = round(accuracy, 2)
        precision_results.loc[precision_results['method'] == current_method, theme] = round(precision, 2)
        recall_results.loc[recall_results['method'] == current_method, theme] = round(recall, 2)
        f1_results.loc[f1_results['method'] == current_method, theme] = round(f1, 2)

        print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')
        # print_confusion_table(y_test_theme, y_pred, current_method, theme)

# Вызов функции
svm_classifier_and_save(X_train, X_test, Y_train, Y_test, theme_list,
                        accuracy_results, precision_results, recall_results, f1_results)

# Сохранение метрик
# save_metrics()


# Часть 5: Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

def bayes_classification(X_train, X_test, Y_train, Y_test, theme_list,
                         accuracy_results, precision_results, recall_results, f1_results):
    current_method = 'Naive Bayes'
    print("\n" + current_method)

    # Преобразование текстов в числовой вид с помощью TF-IDF
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Цикл по всем тематикам (классам)
    for theme in theme_list:
        # Создаем бинарный классификатор для текущей тематики
        classifier = MultinomialNB()

        # Обучение на обучающей выборке
        classifier.fit(X_train_tfidf, Y_train[theme])

        # Предсказание на тестовой выборке
        y_pred = classifier.predict(X_test_tfidf)
        y_true = Y_test[theme].values

        # Расчет метрик
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        # Запись метрик в таблицы (по методу и теме)
        accuracy_results.loc[accuracy_results['method'] == current_method, theme] = round(accuracy, 2)
        precision_results.loc[precision_results['method'] == current_method, theme] = round(precision, 2)
        recall_results.loc[recall_results['method'] == current_method, theme] = round(recall, 2)
        f1_results.loc[f1_results['method'] == current_method, theme] = round(f1, 2)

        print(f'Theme: {theme}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

        # Вывод таблицы с метриками
        print_confusion_table(y_true, y_pred, current_method, theme)


In [ ]:
# Вызов функции классификации
bayes_classification(X_train, X_test, Y_train, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)

# Сохранение метрик в Excel
save_metrics()

# Часть 6: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

def logistic_regression_classification(X_train, X_test, Y_train, Y_test, theme_list,
                                       accuracy_results, precision_results, recall_results, f1_results):
    current_method = 'Logistic Regression'
    print("\n" + current_method)

    # Преобразование текстов в числовой вид с помощью TF-IDF
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Цикл по всем тематикам (классам)
    for theme in theme_list:
        # Создаем бинарный классификатор для текущей тематики
        classifier = LogisticRegression(random_state=42)

        # Обучение на обучающей выборке
        classifier.fit(X_train_tfidf, Y_train[theme])

        # Предсказание на тестовой выборке
        y_pred = classifier.predict(X_test_tfidf)
        y_true = Y_test[theme].values

        # Расчет метрик
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        # Запись метрик в таблицы (по методу и теме)
        accuracy_results.loc[accuracy_results['method'] == current_method, theme] = round(accuracy, 2)
        precision_results.loc[precision_results['method'] == current_method, theme] = round(precision, 2)
        recall_results.loc[recall_results['method'] == current_method, theme] = round(recall, 2)
        f1_results.loc[f1_results['method'] == current_method, theme] = round(f1, 2)

        print(f'Theme: {theme}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}')

        # Вывод таблицы с метриками
        print_confusion_table(y_true, y_pred, current_method, theme)


In [ ]:
# Вызов функции классификации
logistic_regression_classification(X_train, X_test, Y_train, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)

# Сохранение метрик в Excel
save_metrics()

# Запуск всех методов с итерациями и проверкой устойчивости методов (подсчет средних метрик)

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Количество итераций
num_iterations = 3  # Задаем нужное количество итераций
output_dir = '/content/drive/My Drive/Магистерская диссертация/4 семестр/Metrics results'
methods = ['Decision Tree', 'Zero-Shot Binary', 'Zero-Shot Binary Optimal', 'Zero-Shot Multi', 'SVM', 'Naive Bayes', 'Logistic Regression']

# Очистка всех подпапок с итерациями (файлы не удаляются)
def clear_folders(output_dir):
    if os.path.exists(output_dir):
        for folder_name in os.listdir(output_dir):
            folder_path = os.path.join(output_dir, folder_name)
            if os.path.isdir(folder_path):
                for file_name in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, file_name)
                    os.remove(file_path)
                os.rmdir(folder_path)
                print(f"Очищена папка: {folder_path}")
    else:
        os.makedirs(output_dir)
        print(f"Создана основная папка: {output_dir}")

# Функция для создания и очистки папок
def create_folders(output_dir, num_iterations):
    clear_folders(output_dir)
    for i in range(1, num_iterations + 1):
        iteration_dir = os.path.join(output_dir, str(i))
        os.makedirs(iteration_dir, exist_ok=True)

# Функция инициализации таблиц метрик
def initialize_metric_tables(theme_list):
    accuracy_results = pd.DataFrame({'method': methods})
    precision_results = pd.DataFrame({'method': methods})
    recall_results = pd.DataFrame({'method': methods})
    f1_results = pd.DataFrame({'method': methods})

    for theme in theme_list:
        accuracy_results[theme] = 0.0
        precision_results[theme] = 0.0
        recall_results[theme] = 0.0
        f1_results[theme] = 0.0

    return accuracy_results, precision_results, recall_results, f1_results

# Функция сохранения метрик в Excel
def save_metrics(accuracy_results, precision_results, recall_results, f1_results, iteration_dir):
    accuracy_results.to_excel(os.path.join(iteration_dir, 'accuracy.xlsx'), index=False)
    precision_results.to_excel(os.path.join(iteration_dir, 'precision.xlsx'), index=False)
    recall_results.to_excel(os.path.join(iteration_dir, 'recall.xlsx'), index=False)
    f1_results.to_excel(os.path.join(iteration_dir, 'f1.xlsx'), index=False)

# Функция усреднения метрик
def average_metrics(output_dir, num_iterations):
    metrics = ['accuracy', 'precision', 'recall', 'f1']
    avg_results = {}

    # Папка для средних значений
    average_dir = os.path.join(output_dir, 'average')
    os.makedirs(average_dir, exist_ok=True)

    for metric in metrics:
        all_dfs = []
        for i in range(1, num_iterations + 1):
            iteration_dir = os.path.join(output_dir, str(i))
            file_path = os.path.join(iteration_dir, f'{metric}.xlsx')
            df = pd.read_excel(file_path)
            all_dfs.append(df)

        # Усреднение по всем таблицам
        avg_df = round(pd.concat(all_dfs).groupby('method', as_index=False).mean(), 2)
        avg_results[metric] = avg_df

        # Сохранение усредненной таблицы
        # avg_file_path = os.path.join(output_dir, f'average_{metric}.xlsx')
        avg_file_path = os.path.join(average_dir, f'average_{metric}.xlsx')
        avg_df.to_excel(avg_file_path, index=False)

# Создание папок
create_folders(output_dir, num_iterations)

# Выполнение итераций
for iteration in range(1, num_iterations + 1):
    # Генерация случайного random_state
    random_state = np.random.randint(0, 10000)

    # Разделение на обучающую и тестовую выборки (80/20)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=random_state)

    # Инициализация таблиц метрик для текущей итерации
    accuracy_results, precision_results, recall_results, f1_results = initialize_metric_tables(theme_list)

    # Запуск всех методов классификации
    decision_tree_classifier(X_train, X_test, Y_train, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)
    zero_shot_binary_classifier(X_test, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)

    # Вычисляем оптимальные пороги
    optimal_thresholds = find_optimal_thresholds(X_test, Y_test, theme_list)
    # Запускаем классификатор с оптимальными порогами
    zero_shot_binary_classifier_with_thresholds(X_test, Y_test, theme_list, optimal_thresholds, accuracy_results, precision_results, recall_results, f1_results)

    zero_shot_multi_classifier(X_test, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)
    svm_classifier(X_train, X_test, Y_train, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)
    bayes_classification(X_train, X_test, Y_train, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)
    logistic_regression_classification(X_train, X_test, Y_train, Y_test, theme_list, accuracy_results, precision_results, recall_results, f1_results)

    # Сохранение метрик для текущей итерации
    iteration_dir = os.path.join(output_dir, str(iteration))
    save_metrics(accuracy_results, precision_results, recall_results, f1_results, iteration_dir)

    print(f"Завершена {iteration} итерация.")

# Усреднение метрик
average_metrics(output_dir, num_iterations)

print(f"Завершено {num_iterations} итераций с разными random_state.")
print(f"Средние таблицы метрик сохранены в папке {os.path.join(output_dir, 'average')}.")
